In [1]:
import pandas as pd
from datetime import datetime
import mca
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
from sklearn.preprocessing import LabelEncoder

C:\Users\mingt\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Split the file since its too big for github, extract the file outside of .git repository

hpg_res = pd.read_csv('C:\\Users\\mingt\\Downloads\\hpg_reserve.csv\\hpg_reserve.csv')
hpg_res.info()
df1 = hpg_res.iloc[0:100160]
df2 = hpg_res.iloc[100161:200320]
#df1.to_csv('hpg_reserve1.csv')
#df2.to_csv('hpg_reserve2.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


In [3]:
air_reserve = pd.read_csv('air_reserve.csv')
air_store_info= pd.read_csv('air_store_info.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
date_info = pd.read_csv('date_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [9]:
#concat files back after its been uploaded on github
hpg_reserve1 = pd.read_csv('hpg_reserve1.csv')
hpg_reserve2 = pd.read_csv('hpg_reserve2.csv')

hpg_reserve = pd.concat([hpg_reserve1, hpg_reserve2])
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])

In [10]:
air_combined = pd.merge(air_reserve, air_store_info, on='air_store_id', how='outer')
hpg_combined = pd.merge(hpg_reserve, hpg_store_info, on='hpg_store_id', how='left')

air_combined.head(5)
hpg_combined.head(5)

,Unnamed: 0,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,hpg_genre_name,hpg_area_name,latitude,longitude
0,0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,NaN,NaN,NaN,NaN
1,1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,French,Hyōgo-ken Kōbe-shi None,34.692109,135.191698
2,2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,NaN,NaN,NaN,NaN
3,3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,NaN,NaN,NaN,NaN
4,4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,NaN,NaN,NaN,NaN


In [11]:
sub_store = sample_submission['id'].apply(lambda x: str(x).split('_', 2)[:2])
sub_dates = pd.to_datetime(sample_submission['id'].apply(lambda x: str(x).split('_', 2)[2]).rename('Date'))
sub_stores = pd.Series(['_'.join(x) for x in sub_store]).rename('Restaurant')

sub_df = pd.DataFrame({
        'Restaurant': sub_stores,
        'year': sub_dates.dt.year,
        'month': sub_dates.dt.month,
        'day': sub_dates.dt.day })

testing_feats = pd.concat([sample_submission, sub_df], axis=1)

In [12]:
testing_feats

,id,visitors,Restaurant,day,month,year
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,23,4,2017
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,24,4,2017
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,25,4,2017
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,26,4,2017
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,27,4,2017
5,air_00a91d42b08b08d9_2017-04-28,0,air_00a91d42b08b08d9,28,4,2017
6,air_00a91d42b08b08d9_2017-04-29,0,air_00a91d42b08b08d9,29,4,2017
7,air_00a91d42b08b08d9_2017-04-30,0,air_00a91d42b08b08d9,30,4,2017
8,air_00a91d42b08b08d9_2017-05-01,0,air_00a91d42b08b08d9,1,5,2017
9,air_00a91d42b08b08d9_2017-05-02,0,air_00a91d42b08b08d9,2,5,2017


In [41]:
#len(sub_stores.value_counts())
#len(sub_stores.value_counts())

len(testing_feats['Restaurant'].value_counts())

821

In [30]:
hpg_n_store = store_id_relation.merge(hpg_combined, on='hpg_store_id', how='left')

air_n_hpg = air_combined.merge(hpg_n_store, on='air_store_id', how='left')

i = pd.to_datetime(df2['visit_datetime_x'])
df2['all_dates'] = i.dt.date.astype(str)
df2['latitude'] = df2['latitude_x'].astype(str)
df2['longitude'] = df2['longitude_x'].astype(str)
#date_info['calendar_dates'] = pd.to_datetime(date_info['calendar_date'])

date_info['Is_holiday'] = date_info['holiday_flg'].map({1: 'Yes', 0: 'No'})
date_info['Is_weekend'] = date_info['day_of_week'].map({'Saturay': 'Yes' 'Yes', 0: 'No'})
df3 = pd.merge(df2, date_info, how='left', left_on='all_dates', right_on='calendar_date')

In [39]:
df3

,air_store_id,visit_datetime_x,reserve_datetime_x,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id,Unnamed: 0,...,latitude_y,longitude_y,all_dates,latitude,longitude,calendar_date,day_of_week,holiday_flg,Is_holiday,Is_weekend
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-01,35.6580681,139.7515992,2016-01-01,Friday,1.0,Yes,NaN
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-01,35.6580681,139.7515992,2016-01-01,Friday,1.0,Yes,NaN
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-02,35.6580681,139.7515992,2016-01-02,Saturday,1.0,Yes,NaN
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-02,35.6580681,139.7515992,2016-01-02,Saturday,1.0,Yes,NaN
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-08,35.6580681,139.7515992,2016-01-08,Friday,0.0,No,NaN
5,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-08 21:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-08,35.6580681,139.7515992,2016-01-08,Friday,0.0,No,NaN
6,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-05 13:00:00,3.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-08,35.6580681,139.7515992,2016-01-08,Friday,0.0,No,NaN
7,air_877f79706adbfb06,2016-01-09 20:00:00,2016-01-06 23:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-09,35.6580681,139.7515992,2016-01-09,Saturday,0.0,No,NaN
8,air_877f79706adbfb06,2016-01-09 21:00:00,2016-01-09 21:00:00,4.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-09,35.6580681,139.7515992,2016-01-09,Saturday,0.0,No,NaN
9,air_877f79706adbfb06,2016-01-09 22:00:00,2016-01-05 12:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,...,NaN,NaN,2016-01-09,35.6580681,139.7515992,2016-01-09,Saturday,0.0,No,NaN


In [19]:
a = pd.to_datetime(df3['visit_datetime_x'])
b = pd.to_datetime(df3['reserve_datetime_x'])
c = pd.to_datetime(df3['visit_datetime_y'])
d = pd.to_datetime(df3['reserve_datetime_y'])



datetime_df =pd.DataFrame({
        'air_visit_year': a.dt.year,
        'air_visit_month': a.dt.month,
        'air_visit_day': a.dt.day,
        'air_visit_hour': a.dt.hour,
        'air_reserve_year': b.dt.year,
        'air_reserve_month': b.dt.month,
        'air_reserve_day': b.dt.day,
        'air_reserve_hour': b.dt.hour,
         
        'hpg_visit_year': c.dt.year,
        'hpg_visit_month': c.dt.month,
        'hpg_visit_day': c.dt.day,
        'hpg_visit_hour': c.dt.hour,
        'hpg_reserve_year': d.dt.year,
        'hpg_reserve_month': d.dt.month,
        'hpg_reserve_day': d.dt.day,
        'hpg_reserve_hour': d.dt.hour        
    }).fillna(0)



In [29]:
datetime_df

,air_reserve_day,air_reserve_hour,air_reserve_month,air_reserve_year,air_visit_day,air_visit_hour,air_visit_month,air_visit_year,hpg_reserve_day,hpg_reserve_hour,hpg_reserve_month,hpg_reserve_year,hpg_visit_day,hpg_visit_hour,hpg_visit_month,hpg_visit_year
0,1.0,16.0,1.0,2016.0,1.0,19.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,16.0,1.0,2016.0,1.0,20.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,16.0,1.0,2016.0,2.0,18.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,16.0,1.0,2016.0,2.0,21.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,20.0,1.0,2016.0,8.0,21.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8.0,21.0,1.0,2016.0,8.0,21.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,13.0,1.0,2016.0,8.0,21.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6.0,23.0,1.0,2016.0,9.0,20.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,21.0,1.0,2016.0,9.0,21.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5.0,12.0,1.0,2016.0,9.0,22.0,1.0,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
for i in categorical_df:
    print(len(categorical_df[i].value_counts()))

829
14
103
108
108
16
33


In [20]:
categorical_df = df3[['air_store_id', 'air_genre_name', 'air_area_name', 
                    'latitude', 'longitude','hpg_genre_name', 
                      'hpg_area_name']].fillna('None_Stated').apply(LabelEncoder().fit_transform)

#dummy_df = pd.get_dummies(df3[['day_of_week', 'Is_holiday']])

#Over a million dummy variables, too much
#dummy_df = pd.get_dummies(categorical_df)
features = pd.concat([datetime_df, categorical_df], axis=1)

air_visitors = df3['reserve_visitors_x'].fillna(0)
hpg_visitors = df3['reserve_visitors_y'].fillna(0)
df3['reserve_total'] = air_visitors + hpg_visitors

target = df3['reserve_total']

In [42]:
features.info()
s.features['air_store_id']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171047 entries, 0 to 1171046
Data columns (total 23 columns):
air_reserve_day      1170265 non-null float64
air_reserve_hour     1170265 non-null float64
air_reserve_month    1170265 non-null float64
air_reserve_year     1170265 non-null float64
air_visit_day        1170265 non-null float64
air_visit_hour       1170265 non-null float64
air_visit_month      1170265 non-null float64
air_visit_year       1170265 non-null float64
hpg_reserve_day      1112519 non-null float64
hpg_reserve_hour     1112519 non-null float64
hpg_reserve_month    1112519 non-null float64
hpg_reserve_year     1112519 non-null float64
hpg_visit_day        1112519 non-null float64
hpg_visit_hour       1112519 non-null float64
hpg_visit_month      1112519 non-null float64
hpg_visit_year       1112519 non-null float64
air_store_id         1171047 non-null int64
air_genre_name       1171047 non-null int64
air_area_name        1171047 non-null int64
latitude           

0          426
1          426
2          426
3          426
4          426
5          426
6          426
7          426
8          426
9          426
10         426
11         426
12         426
13         426
14         426
15         426
16         426
17         426
18         426
19         426
20         426
21         426
22         426
23         426
24         426
25         426
26         426
27         426
28         426
29         426
          ... 
1171017     20
1171018    184
1171019    272
1171020    129
1171021    580
1171022    780
1171023    376
1171024    524
1171025    140
1171026    315
1171027    547
1171028    648
1171029    437
1171030    782
1171031    197
1171032    409
1171033     98
1171034    280
1171035    105
1171036    262
1171037     17
1171038    383
1171039    433
1171040    707
1171041    756
1171042    796
1171043    215
1171044    509
1171045    375
1171046    654
Name: air_store_id, Length: 1171047, dtype: int64

In [27]:
df3.groupby(by='air_store_id')

In [61]:
#X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                    #test_size=0.1, random_state=7)



dmatrix = xgb.DMatrix(features, target)

xgb_params_cv = dict(eta=0.1,
                  objective='reg:linear',
                  eval_metric='rmse')
best = xgb.cv(params = xgb_params_cv, dtrain=dmatrix, num_boost_round=1000, early_stopping_rounds=100, verbose_eval=True)

best.tail(5)               

[0]	train-rmse:8.86682+0.0158384	test-rmse:8.86695+0.0349489
[1]	train-rmse:8.37948+0.0163708	test-rmse:8.37972+0.0346245
[2]	train-rmse:7.96216+0.0167403	test-rmse:7.96237+0.034431
[3]	train-rmse:7.60529+0.0174889	test-rmse:7.60586+0.0337274
[4]	train-rmse:7.3046+0.0179879	test-rmse:7.30527+0.0331708
[5]	train-rmse:7.04977+0.0192915	test-rmse:7.05059+0.0316244
[6]	train-rmse:6.83669+0.0190606	test-rmse:6.83741+0.0316507
[7]	train-rmse:6.64391+0.032193	test-rmse:6.64471+0.0188135
[8]	train-rmse:6.47997+0.0274074	test-rmse:6.48152+0.0292925
[9]	train-rmse:6.33845+0.0341722	test-rmse:6.34064+0.0208445
[10]	train-rmse:6.21602+0.0295812	test-rmse:6.219+0.0210086
[11]	train-rmse:6.11264+0.0251445	test-rmse:6.11568+0.0263122
[12]	train-rmse:6.014+0.0250338	test-rmse:6.01687+0.0250242
[13]	train-rmse:5.93239+0.0267482	test-rmse:5.9362+0.023856
[14]	train-rmse:5.86469+0.0208583	test-rmse:5.86959+0.0283047
[15]	train-rmse:5.80857+0.0277799	test-rmse:5.81419+0.0210877
[16]	train-rmse:5.75348+0.0

KeyboardInterrupt: 